In [2]:
import pandas as pd

In [4]:
rides = pd.read_csv("2011-capitalbikeshare-tripdata.zip")
rides.info()

FileNotFoundError: [Errno 2] No such file or directory: '2011-capitalbikeshare-tripdata.zip'

In [ ]:
rides.head()

In [ ]:
del rides["Start station"]
del rides["End station"]

In [ ]:
rides['Member type'].value_counts()

In [ ]:
rides = rides[rides['Member type'] != "Unknown"]

In [ ]:
rides['Member type'].value_counts()

In [ ]:
rides['start'] = pd.to_datetime(rides['Start date'])
rides['start'].iloc[0]

In [ ]:
rides['start'].tail(10).dt.strftime("%Y-%m-%d")

In [ ]:
rides['date'] = rides['start'].dt.date#strftime("%Y-%m-%d")
rides['hour'] = rides['start'].dt.hour

In [ ]:
grouped_as_index = rides.groupby(['date', 'hour', 'Member type']).size()
rides_by_hour = grouped_as_index.to_frame("rides").reset_index()
rides_by_hour.head()

In [ ]:
from pandas.tseries.holiday import USFederalHolidayCalendar

In [ ]:
import datetime

In [ ]:
holidays = pd.DataFrame({
    'date': USFederalHolidayCalendar().holidays(datetime.date(2011,1,1), datetime.date(2015,12,31)).date,
    'is_holiday': True})
holidays.head()

In [ ]:
# too few
rides_by_hour_with_holidays = pd.merge(
    rides_by_hour,
    holidays,
    left_on='date',
    right_on='date')
print(len(rides_by_hour_with_holidays))
rides_by_hour_with_holidays.head()

In [ ]:
rides_by_hour_with_holidays = pd.merge(
    rides_by_hour,
    holidays,
    left_on='date',
    right_on='date',
    how='left',
    indicator=False)
print(len(rides_by_hour_with_holidays))
rides_by_hour_with_holidays.head()

In [ ]:
rides_by_hour_with_holidays['is_holiday'] = rides_by_hour_with_holidays['is_holiday'].fillna(False)

In [ ]:
rides_by_hour_with_holidays.head()

https://www.ncei.noaa.gov/data/global-hourly/doc/isd-format-document.pdf
https://www.ncei.noaa.gov/data/global-hourly/doc/CSV_HELP.pdf

There's a "Find a Station" tool, but it's confusing how to use the results. https://www.ncdc.noaa.gov/data-access/land-based-station-data/station-metadata has a link to a [station list file](ftp://ftp.ncdc.noaa.gov/pub/data/noaa/isd-history.txt). Searching that, it looks like the code for Reagan Airport is 724050 13743. So the file is
https://www.ncei.noaa.gov/data/global-hourly/access/2011/72405013743.csv

In [ ]:
weather = pd.read_csv("https://www.ncei.noaa.gov/data/global-hourly/access/2011/72405013743.csv")

In [ ]:
weather['TMP'].str.split(',', n=1, expand=True).head()

In [ ]:
weather[['temp_raw', 'temp_validity']] = weather['TMP'].str.split(',', n=1, expand=True)
weather[['temp_raw', 'temp_validity']].head()

In [ ]:
weather['temp_validity'].value_counts(dropna=False)

Meanings:

* 5: Passed QC
* 1: Passed QC
* 9: Outside of limits
* A: suspect but good


In [ ]:
weather[weather.temp_validity == "A"].temp_raw

In [ ]:
weather['temp_is_valid'] = weather['temp_validity'].isin(["5", "1", "A"])
weather = weather[weather['temp_is_valid']].copy()

In [ ]:
weather['temp_C'] = pd.to_numeric(weather['temp_raw']) / 10

In [ ]:
hourly_temp_data = weather.resample('1H', on='date')['temp_C'].mean().reset_index()
hourly_temp_data.head()

In [ ]:
hourly_temp_data['hour'] = hourly_temp_data['date'].dt.hour
hourly_temp_data['date'] = hourly_temp_data['date'].dt.date
hourly_temp_data.head()

In [ ]:
merged_data = pd.merge(
    rides_by_hour_with_holidays,
    hourly_temp_data,
    on=['date', 'hour'])
merged_data.head()

In [ ]:
assert len(merged_data) > 365 * 23
assert 'date' in merged_data.columns
assert 'hour' in merged_data.columns
assert 'is_holiday' in merged_data.columns
assert 'temp_C' in merged_data.columns
assert 'rides' in merged_data.columns
assert not any(merged_data.isna())

In [ ]:
import seaborn as sns
%matplotlib inline

In [ ]:
sns.jointplot(x='hour', y='rides', data=merged_data, kind='hex')

In [ ]:
merged_data.groupby('hour').rides.mean().plot()